In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

def trực quan hoá dữ liệu dự báo

In [2]:
def Visualize_model_reg(y, y_pred):
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.scatter(y_pred, y)
    plt.xlabel('Model Prediction')
    plt.ylabel('True Vale')
    plt.plot([0, np.max(y) + 2*np.min(y)], [0, np.max(y) + 2*np.min(y)], '-', color = 'r')
    plt.subplot(1,2,2)
    sns.distplot(y, hist=False, color='r', label='True Value')
    sns.distplot(y_pred, hist=False, color='b', label='Model Prediction', axlabel='Distribution')
    plt.show()

In [3]:
def Visualize_model_reg2(y_train, y_test, y_train_hat, y_test_hat):
    plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    ax1 = sns.distplot(y_train, hist=False, color="b", label='Train Actual')
    sns.distplot(y_train_hat, hist=False, color="r", label='Train Predict', ax=ax1)
    plt.subplot(1,2,2)
    ax2 = sns.distplot(y_test, hist=False, color="b", label='Test Actual')
    sns.distplot(y_test_hat, hist=False, color="r", label='Test Predict', ax=ax2)
    plt.show()

def tính toán so sánh kết quả dự báo và dữ liệu thật

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def Static_score_model_reg(y, y_pred):
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    return r2, mse, mae

Ghi model

In [5]:
# lưu model và load model
# Ap dung duoc voi ca model dung trong feature engineering va model du doan cuoi cung
def Save_Object(obj, filename):
    import pickle
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)
    return
def Load_Object(filename):
    import pickle
    with open(filename, 'rb') as file:
        obj = pickle.load(file)
    return obj

Kiểm tra hàm định nghĩa

In [6]:
# df = pd.read_csv('Data/Test_data.csv')
# df.head()

In [7]:
# Visualize_model_reg(df['Actual Value'], df['Predict Value'])

In [8]:
# r2, mse, mae = Static_score_model_reg(df['Actual Value'], df['Predict Value'])
# print('Model score: ', r2)
# print('Model MSE: ', mse)
# print('Model MAE: ', mae)

class của project

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
class Pre_Processing(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('Khởi tạo đối tượng Pre_Processing')
    def fit(self, X, y=None):
#         print('Vào lệnh Fit Pre_Process()')
        return self
    def transform(self, data, y=None):
        df = data.copy()
        df = addvar_gia(df)
        df = addvar_dongia(df)
        df = addvar_nhavesinh(df)
        df = addvar_sophong(df)
        df = addvar_cautruc_binary(df)
        df = addvar_cautruc_num(df)
        df = addvar_dientich(df)
        df = addvar_dientichsudung(df)
        df = addvar_kichthuoc(df)
        df = addvar_logioi(df)
        df = addvar_location(df)
        df = addvar_loaibds(df)
        return df

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
class Create_XY(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('Khởi tạo đối tượng Create_XY')
    def fit(self, X, y=None):
        return self
    def transform(self, data, y=None):
        df = data.copy()
        X, y = select_var(df)
        return X, y

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
class Concating(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('Khởi tạo đối tượng Concating')
    def fit(self, X, y=None):
        return self
    def transform(self, data, y=None):
        import pickle
        with open('model_ohe_buithihuong.pkl', 'rb') as file:
            encoder = pickle.load(file)
        
        df = data.copy()
        lst_encode = [ 'f_Quan', 'f_Location_qa_2']
        arr = encoder.transform(df[lst_encode]).toarray()

        cols = []
        n = 0
        for i in encoder.categories_:
            for j in i[0:]: 
                t = 'oh_' + lst_encode[n] + '_' +str(j)
                t = t.replace('-', '_')
                cols.append(t)
            n = n+1
        X_new_oh_encode = pd.DataFrame(arr, columns=cols)
        
        lst_concat = ['f_Nha_ve_sinh', 'f_Chieu_dai', 'f_Rong', 'f_Dien_tich', 'f_Dien_tich_su_dung', 
                      'f_Lo_gioi', 'f_Lau_n', 'f_Nha_hem']
        X_new = pd.concat([X_new_oh_encode, df[lst_concat]], axis=1)
        X_new.head()     
        
        return X_new

def của project House pricing

In [12]:
def no_accent_vietnamese(s):
    import re
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    return s

In [13]:
def continous_feature_describe(lst_cols, df):
    col_ = []
    count_ = []
    min_ = []
    max_ = []
    mean_ = []
    std_ = []
    var_ = []
    skewness_ = []
    kurtosis_ = []
    do_lech = []
    do_nhon = []

    for i in lst_cols:
        col_.append(i)
        count_.append(df[i].count())
        min_.append(df[i].min())
        max_.append(df[i].max())
        mean_.append(df[i].mean())
        std_.append(df[i].std())
        var_.append(df[i].var())
        skewness_.append(df[i].skew())
        kurtosis_.append(df[i].kurtosis())
        
        if df[i].skew() > 0: do_lech.append('lech phai')
        elif df[i].skew() < 0: do_lech.append('lech trai')
        else: do_lech.append('doi xung')
            
        if df[i].kurtosis() > 0: do_nhon.append('nhon hon PP chuan')
        elif df[i].kurtosis() < 0: do_nhon.append('bet hon PP chuan')
        else: do_nhon.append('tuong duong PP chuan')

    df_describe = pd.DataFrame({'feature_name': col_,
                               'count': count_,
                               'min': min_,
                               'max': max_,
                               'mean': mean_,
                               'std': std_,
                               'var': var_,
                                'skewness': skewness_,
                                'kurtosis': kurtosis_,
                                'do_lech': do_lech,
                                'do_nhon': do_nhon
                               })
    return df_describe

In [14]:
def addvar_gia(data):
    df = data.copy()
    # Tách dữ liệu về giá trong chuỗi ký tự
    lst_gia = []
    for i in df['Gia'].str.split(" "):
        lst_gia.append(i[0])

    df['f_Gia'] = lst_gia

    # Thay ký tự ',' bằng '.'
    df['f_Gia'] = df['f_Gia'].str.replace(',','.')

    # Kiểm tra kiểu dữ liệu của từng giá trị đã đúng hay chưa
    
    df_gia = df['Gia'].str.split(" ", expand=True)
    lst_gia = []
    n=0
    for i in df['f_Gia']:
        try: float(i)        
        except: lst_gia.append(i)
        else: 
            if df_gia.iloc[n,1] == 'tỷ': lst_gia.append(float(i)*1000)
            elif df_gia.iloc[n,1] == 'triệu': lst_gia.append(float(i))
            else: lst_gia.append(i)
        n=n+1

    df['f_Gia'] = lst_gia

    # Kiểm tra kiểu dữ liệu của từng giá trị đã đúng hay chưa
    lst_gia_check = []
    for i in df['f_Gia']:
        if type(i) is float or type(i) is int: lst_gia_check.append('Y')
        else: lst_gia_check.append('N')

    df_gia_check = pd.DataFrame(lst_gia_check)
    df_gia_check.value_counts()

    # Lọc những giá trị có kiểu dữ liệu chưa đúng
    df.iloc[df_gia_check[df_gia_check[0] == 'N'].index, df.columns.to_list().index('f_Gia')].values

    # Tạm thời thay thế các thành NaN
    df.iloc[df_gia_check[df_gia_check[0] == 'N'].index, df.columns.to_list().index('f_Gia')] = np.nan
    
    # Chuẩn hoá kiểu dữ liệu
    df['f_Gia'] = df['f_Gia'].astype(float)
    
    return df

In [15]:
def addvar_dongia(data):
    df = data.copy()
    lst_dongia = []
    for i in df['Gia'].str.split(" "):
        lst_dongia.append(i[2])

    df['f_Don_gia'] = lst_dongia
    df['f_Don_gia'] = df['f_Don_gia'].str.replace(',','.')
    df['f_Don_gia'] = df['f_Don_gia'].astype(float)
    df.loc[df['Gia'].str.contains('tỷ/m'),'f_Don_gia'] = df.loc[df['Gia'].str.contains('tỷ/m'),'f_Don_gia']*1000
    df.loc[df['Gia'].str.contains('ngàn/m'),'f_Don_gia'] = df.loc[df['Gia'].str.contains('ngàn/m'),'f_Don_gia']/1000
    return df

In [16]:
def addvar_dientich(data):
    df=data.copy()
    # Kiểm tra kiểu dữ liệu của từng giá trị đã đúng hay chưa
    lst_dt_check = []
    for i in df['Dien_tich']:
        if type(i) is float or type(i) is int: lst_dt_check.append('Y')
        else: lst_dt_check.append('N')

    df_dt_check = pd.DataFrame(lst_dt_check)
    df_dt_check.value_counts()

    # Lọc những giá trị có kiểu dữ liệu chưa đúng
    lst_dt_to_replace = df.iloc[df_dt_check[df_dt_check[0] == 'N'].index, df.columns.to_list().index('Dien_tich')].values
    lst_dt_to_replace
    
    df['f_Dien_tich'] = df['Dien_tich']
    if len(lst_dt_to_replace) > 0: 
        df.loc[df['f_Dien_tich'].str.count('.')>=2,'f_Dien_tich'] = df.loc[df['f_Dien_tich'].str.count('.')>=2,'f_Dien_tich']\
                                                                .str.replace('.','',1)
    # Chuyển kiểu dữ liệu về dạng số liên tục
    df['f_Dien_tich'] = df['f_Dien_tich'].astype(float)    
   
    # Fill NaN của f_Gia bằng f_Dien_tich và f_Don_gia
    df.loc[df['f_Gia'].isna(), 'f_Gia'] = df.loc[df['f_Gia'].isna(), 'f_Dien_tich'] * df.loc[df['f_Gia'].isna(), 'f_Don_gia']

    return df

In [17]:
def addvar_dientichsudung(data):
    df=data.copy()
    # Kiểm tra kiểu dữ liệu của từng giá trị đã đúng hay chưa
    lst_dtsd_check = []
    for i in df['Dien_tich_su_dung']:
        if type(i) is float or type(i) is int: lst_dtsd_check.append('Y')
        else: lst_dtsd_check.append('N')

    df_dtsd_check = pd.DataFrame(lst_dtsd_check)
    df_dtsd_check.value_counts()

    # Lọc những giá trị có kiểu dữ liệu chưa đúng
    lst_dtsd_to_replace = df.iloc[df_dtsd_check[df_dtsd_check[0] == 'N'].index, df.columns.to_list().index('Dien_tich_su_dung')].values
    lst_dtsd_to_replace

    df['f_Dien_tich_su_dung'] = df['Dien_tich_su_dung']
    if len(lst_dtsd_to_replace) > 0: 
        df.loc[df['f_Dien_tich_su_dung'].str.count('.')>=2,'f_Dien_tich_su_dung'] = df.loc[df['f_Dien_tich_su_dung'].str.count('.')>=2,'f_Dien_tich_su_dung']\
                                                                .str.replace('.','',1)

    # Chuyển kiểu dữ liệu về dạng số liên tục
    df['f_Dien_tich_su_dung'] = df['f_Dien_tich_su_dung'].astype(float)
    
    # Thay thế giá trị bị lỗi typing (diện tích sử dụng > diện tích 50 lần)
    df.loc[((df['f_Dien_tich_su_dung']/df['f_Dien_tich'])>=50) & (df['f_Dien_tich']<=2), 'f_Dien_tich'] = \
        df.loc[((df['f_Dien_tich_su_dung']/df['f_Dien_tich'])>=50) & (df['f_Dien_tich']<=2), 'f_Dien_tich'] * 1000
    
    return df

In [18]:
def addvar_nhavesinh(data):
    df = data.copy()
    # Kiểm tra kiểu dữ liệu của từng giá trị đã đúng hay chưa
    lst_nha_ve_sinh = []
    for i in df['Nha_ve_sinh']:
        try: int(i)        
        except: lst_nha_ve_sinh.append(i)
        else: lst_nha_ve_sinh.append(int(i))

    df['f_Nha_ve_sinh'] = lst_nha_ve_sinh

    # Kiểm tra kiểu dữ liệu của từng giá trị đã đúng hay chưa
    lst_nvs_check = []
    for i in df['f_Nha_ve_sinh']:
        if type(i) is int: lst_nvs_check.append('Y')
        else: lst_nvs_check.append('N')

    df_nvs_check = pd.DataFrame(lst_nvs_check)
    df_nvs_check.value_counts()

    # Lọc những giá trị có kiểu dữ liệu chưa đúng
    df.iloc[df_nvs_check[df_nvs_check[0] == 'N'].index, df.columns.to_list().index('f_Nha_ve_sinh')].values

    # Tạm thời thay thế các thành NaN
    df.iloc[df_nvs_check[df_nvs_check[0] == 'N'].index, df.columns.to_list().index('f_Nha_ve_sinh')] = np.nan
#    df.loc[df['f_Nha_ve_sinh'].isna(), 'f_Nha_ve_sinh'] = round(df.loc[df['f_Nha_ve_sinh'].isna()==False, 'f_Dien_tich'] / 28.5, 0) #==> Them moi
    df.loc[df['f_Nha_ve_sinh'].isna(), 'f_Nha_ve_sinh'] = df.loc[df['f_Nha_ve_sinh'].isna()==False, 'f_Nha_ve_sinh'].mean() 
        
    # Chuẩn hoá kiểu dữ liệu
#     df['f_Nha_ve_sinh'] = df['f_Nha_ve_sinh'].astype('Int64')
    df['f_Nha_ve_sinh'] = df['f_Nha_ve_sinh'].astype(int)
    return df

In [19]:
def addvar_sophong(data):
    df=data.copy()
    # Kiểm tra kiểu dữ liệu của từng giá trị đã đúng hay chưa
    lst_so_phong = []
    for i in df['So_phong']:
        try: int(i)        
        except: lst_so_phong.append(i)
        else: lst_so_phong.append(int(i))

    df['f_So_phong'] = lst_so_phong

    lst_sp_check = []
    for i in df['f_So_phong']:
        if type(i) is int: lst_sp_check.append('Y')
        else: lst_sp_check.append('N')

    df_sp_check = pd.DataFrame(lst_sp_check)
    df_sp_check.value_counts()

    # Lọc những giá trị có kiểu dữ liệu chưa đúng
    df.iloc[df_sp_check[df_sp_check[0] == 'N'].index, df.columns.to_list().index('f_So_phong')].values

    # Tạm thời thay thế các thành NaN
    df.iloc[df_sp_check[df_sp_check[0] == 'N'].index, df.columns.to_list().index('f_So_phong')] = np.nan
    
     # Chuẩn hoá kiểu dữ liệu
    df['f_So_phong'] = df['f_So_phong'].astype('Int64')

    return df

In [20]:
def addvar_cautruc_binary(data):
    df=data.copy()
    df['f_Tang_ham_b'] = 0
    df.loc[df['Cau_truc'].str.contains('tầng hầm')==True,'f_Tang_ham_b']=1

    df['f_Gac_suot_b'] = 0
    df.loc[df['Cau_truc'].str.contains('gác suốt')==True,'f_Gac_suot_b']=1

    df['f_Lau_b'] = 0
    df.loc[df['Cau_truc'].str.contains('lầu')==True,'f_Lau_b']=1

    df['f_Lung_b'] = 0
    df.loc[df['Cau_truc'].str.contains('lửng')==True,'f_Lung_b']=1

    df['f_Tret_b'] = 0
    df.loc[df['Cau_truc'].str.contains('trệt')==True,'f_Tret_b']=1

    df['f_Tang_thuong_b'] = 0
    df.loc[df['Cau_truc'].str.contains('tầng thượng')==True,'f_Tang_thuong_b']=1

    df['f_Ap_mai_b'] = 0
    df.loc[df['Cau_truc'].str.contains('áp mái')==True,'f_Ap_mai_b']=1
    return df

In [21]:
def addvar_cautruc_num(data):
    df=data.copy()
    df['f_Tang_ham_n'] = 0
    df.loc[df['Cau_truc'].str.contains('có tầng hầm'),'f_Tang_ham_n'] = 1

    df['f_Gac_suot_n'] = df['Cau_truc'].str.extract('(\d+ gác suốt)', expand = True)[0].str.split(' ', expand=True)[0].fillna(0)
    df['f_Gac_suot_n'] = df['f_Gac_suot_n'].astype(int)

    df['f_Lau_n'] = df['Cau_truc'].str.extract('(\d+ lầu)', expand = True)[0].str.split(' ', expand=True)[0].fillna(0)
    df['f_Lau_n'] = df['f_Lau_n'].astype(int)

    df['f_Lung_n'] = df['Cau_truc'].str.extract('(\d+ lửng)', expand = True)[0].str.split(' ', expand=True)[0].fillna(0)
    df['f_Lung_n'] = df['f_Lung_n'].astype(int)

    df['f_Tret_n'] = df['Cau_truc'].str.extract('(\d+ trệt)', expand = True)[0].str.split(' ', expand=True)[0].fillna(0)
    df['f_Tret_n'] = df['f_Tret_n'].astype(int)

    df['f_Tang_thuong_n'] = df['Cau_truc'].str.extract('(\d+ tầng thượng)', expand = True)[0].str.split(' ', expand=True)[0].fillna(0)
    df['f_Tang_thuong_n'] = df['f_Tang_thuong_n'].astype(int)

    df['f_Ap_mai_n'] = df['Cau_truc'].str.extract('(\d+ áp mái)', expand = True)[0].str.split(' ', expand=True)[0].fillna(0)
    df['f_Ap_mai_n'] = df['f_Ap_mai_n'].astype(int)
    return df

In [22]:
def addvar_kichthuoc(data):
    df=data.copy()
    df['Chieu_dai'].isna().sum(), (df['Chieu_dai']==0).sum()
    df['f_Chieu_dai'] = df['Chieu_dai'].astype(float)

    df['Rong'].isna().sum(), (df['Rong']==0).sum()
    df['f_Rong'] = df['Rong'].astype(float)
    return df

In [23]:
def addvar_logioi(data):
    df=data.copy()
    df['f_Nha_hem'] = 1
    df.loc[df['Hem_rong']==0, 'f_Nha_hem'] = 0
    df['f_Nha_mat_tien'] = 1
    df.loc[df['Duong_mat_tien']==0, 'f_Nha_mat_tien'] = 0
    df['f_Lo_gioi'] = df['Hem_rong'] + df['Duong_mat_tien']
    return df

In [24]:
def addvar_location(data):
    df=data.copy()
    lst_duong = []
    for i in df['Duong']:
        lst_duong.append(no_accent_vietnamese(str.strip(str(i))))
    df['f_Duong'] = lst_duong

    df[df['f_Duong'].str.contains("\?")]['f_Duong'].unique()
    df['f_Duong'] = df['f_Duong'].replace(['???ng Nguy?n Th?n Hi?n', '???ng Cach M?ng Thang Tam','???ng Vo V?n Van'],
                               ['Nguyen Than Hien', 'Duong Cach Mang Thang Tam', 'Duong Vo Van Tan'])
    
    lst_phuong = []
    for i in df['Phuong']:
        lst_phuong.append(no_accent_vietnamese(str.strip(str(i))))
    df['f_Phuong'] = lst_phuong


    lst_quan = []
    for i in df['Quan']:
        lst_quan.append(no_accent_vietnamese(str.strip(i)))
    df['f_Quan'] = lst_quan
    
    location = pd.read_csv('location.csv')    
    df = df.merge(location, left_on=['f_Phuong', 'f_Quan'], 
                  right_on=['Phuong_bin', 'Quan_bin'],
                 how='left')
    df = df.drop(columns=[ 'Phuong_bin', 'Quan_bin'])    
    df['f_Location_qa'] = df['f_Quan'] + '_' +  df['f_Area']
    
    df.loc[df['f_Rank_phuong'].isna(), 'f_Rank_phuong'] = 0
    df.loc[df['f_Rank_quan'].isna(), 'f_Rank_quan'] = 0
    
    
    location_2 = pd.read_excel('location_2.xlsx', sheet_name='bin_phuong')    
    df = df.merge(location_2, left_on=['f_Phuong', 'f_Quan'], 
                  right_on=['Phuong_bin', 'Quan_bin'],
                 how='left')
    df = df.drop(columns=[ 'Phuong_bin', 'Quan_bin'])    
    df['f_Location_qa_2'] = df['f_Quan'] + '_' +  df['f_Area_2']   
    
    
    return df

In [25]:
def addvar_loaibds(data):
    df=data.copy()
    
    lst_noidung = []
    for i in df['Noi_dung']:
        lst_noidung.append(no_accent_vietnamese(str.strip(i)))

    # Tạo thuộc tính 'Quan_1'
    df['Noi_dung_clean'] = lst_noidung
    df['Noi_dung_clean'] = df['Noi_dung_clean'].str.lower()

    lst_word = ['nha', 'tro', 'dat', 'biet thu', 'shophouse', 'villa', 'khach san', 'village', 'nen', 'can ho', 'chung cu', 'can']
    lst_loai_bds = ['nha', 'tro', 'dat', 'biet thu', 'shophouse', 'villa', 'khach san', 'villa', 'dat', 'chung cu', 'chung cu', 'chung cu']

    df['f_Loai_BDS'] = 'other'
    n = 0
    for i in lst_word:
        df.loc[(df['Noi_dung_clean'].str.contains(i)) & (df['f_Loai_BDS']=='other'),'f_Loai_BDS']=lst_loai_bds[n]
        n=n+1
    return df

In [26]:
def select_var(data):
    df = data.copy()
    sample = df[['f_Nha_ve_sinh', 'f_Chieu_dai',  'f_Rong', 'f_Dien_tich', 'f_Dien_tich_su_dung', 
                     'f_Lo_gioi', 'f_Lau_n', 'f_Nha_hem', 
                     'f_Quan', 'f_Location_qa_2', 'f_Gia']]  
#     sample = sample.dropna(how='any').reset_index(drop=True)
    X_new = sample.drop('f_Gia', axis=1)
    y_new = sample['f_Gia']
    
    return X_new, y_new